In [4]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
from config import postgres_pwd

In [5]:
#Extract CSV into DataFrame
prime_file = "./Resources/amazon_prime_movies.csv"
prime_df = pd.read_csv(prime_file)
prime_df.head()

,Movie Name,Language,IMDb Rating,Running Time,Year of Release,Maturity Rating,Plot
0,John Rambo,Tamil,7,1 h 26 min,2008,18+,"In Thailand, John Rambo joins a group of merce..."
1,American Pie,English,7,1 h 35 min,1999,18+,You'll never look at warm apple pie the same w...
2,Bombshell,English,6.8,1 h 48 min,2019,18+,"The provocative real story of three smart, amb..."
3,Love Birds,Tamil,5.1,2 h 40 min,1996,All,"After accidentally meeting, Mridula (Nagma) an..."
4,Hippi,Telugu,5,2 h 20 min,2019,18+,Hippi is about the live-in relationship betwee...


In [8]:
#Creating new Dataset with selected coloumns and clean it
new_prime_df = prime_df[['Movie Name', 'Language', 'IMDb Rating','Maturity Rating']].copy()
new_prime_df.head()

,Movie Name,Language,IMDb Rating,Maturity Rating
0,John Rambo,Tamil,7,18+
1,American Pie,English,7,18+
2,Bombshell,English,6.8,18+
3,Love Birds,Tamil,5.1,All
4,Hippi,Telugu,5,18+


In [10]:
#Transform Prime DataFrame
#Create a filtered dataframe from specific columns
prime_cols = ["Movie Name", "Language", "IMDb Rating", "Maturity Rating"]
prime_transformed = prime_df[prime_cols].copy()

# Rename the column headers
prime_transformed = prime_transformed.rename(columns={"Movie Name": "Movie Title",
                                                          "Language": "Language",
                                                          "IMDb Rating": "IMDB Rating",
                                                        "Maturity Rating": "PG Rating"})

# Clean the data by dropping duplicates and setting the index
prime_transformed.drop_duplicates("Movie Title", inplace=True)
prime_transformed.set_index("IMDB Rating", inplace=True)

prime_transformed.head()


,Movie Title,Language,PG Rating
IMDB Rating,,,
7,John Rambo,Tamil,18+
7,American Pie,English,18+
6.8,Bombshell,English,18+
5.1,Love Birds,Tamil,All
5,Hippi,Telugu,18+


In [15]:
#Create Database connection
connection_string = "postgres:postgres@localhost:5432/prime1_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
# Confirm tables
engine.table_names()

/var/folders/s3/_tx_vh_n3jxcv5xmm_5v0lpr0000gn/T/ipykernel_4293/3015872754.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['prime_table']

In [ ]:
#Use pandas to Load  csv converted DataFrames into database
prime_transformed.to_sql(name='prime_table', con=engine, if_exists='append', index=True)

In [ ]:
#confirm data has been added by quering the table
pd.read_sql_query('select * from prime_table', con=engine).head()